<a href="https://colab.research.google.com/github/naokityokoyama/NLP_Embedding_Transformers/blob/main/NLP_Spacy_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Lib Spacy

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download pt_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 25.6 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.1/457.1 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uni

## Import Lib

In [ ]:
import spacy
import pandas as pd
from tqdm import tqdm
import re
import numpy as np
import requests
from tqdm import tqdm

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Load Database

In [ ]:
train = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Portuguese-Bosque/master/pt_bosque-ud-train.conllu'
dev = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Portuguese-Bosque/master/pt_bosque-ud-dev.conllu'
test = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Portuguese-Bosque/master/pt_bosque-ud-test.conllu'

In [ ]:
response = requests.get(dev)
url = response.text

In [ ]:
class Coleta:
  def __init__(self, url):
    self.url = url
    
  def extrair(self, tamanho = len(url.split('\n'))):
    data_1 = []
    data_2 = []
    data_3 = []
    for i in tqdm(range(tamanho)):  
      data1 = self.url.split('\n')[i].split('\t')
      try:
        if (data1[0][0])!= '#':
          data_1.append(data1[0])
          data_2.append(data1[2])
          data_3.append(data1[3])
      except:
        pass    
    return data_1, data_2, data_3

  def dataframe(self,  tamanho=len(url.split('\n'))):
      
    lista1 = self.extrair(tamanho=tamanho)[0]
    lista2 = self.extrair(tamanho=tamanho)[1]
    lista3 = self.extrair(tamanho=tamanho)[2]
    df = pd.concat([pd.DataFrame(lista1), pd.DataFrame(lista2), pd.DataFrame(lista3)], axis=1)       
    df.columns = ['id', 'token', 'POS_']
    return df
    
  

In [ ]:
dados = Coleta(url)
df = dados.dataframe()

100%|██████████| 35443/35443 [02:56<00:00, 200.55it/s]


## Load Model Spacy pt small

In [ ]:
nlp = spacy.load("pt_core_news_lg")
#df = pd.read_csv('/content/pos_full.csv')

In [ ]:
# def Regex(val):
#   reg = (re.findall(r'\w([,])', val))
#   if reg:
#       return (val.replace(',', ''))
#   else:
#     return val   

# df['token'] = df['token'].apply(lambda x: Regex(x))
# df['token'] = df['token'].str.replace('/', '')

In [ ]:
def valor(x):
  puctuation = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
  if x not in puctuation:
    return x

df['token'] = df['token'].apply(lambda x: valor(x))

df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
lista_POS = []
lista_token = []
for token in tqdm(range(len(df))):
  token_nlp = nlp(df['token'].values[token])
  for pos in token_nlp:
    lista_POS.append(pos.pos_)
    lista_token.append(pos.text)

100%|██████████| 25278/25278 [02:09<00:00, 195.12it/s]


In [ ]:
df_spacy = pd.DataFrame({'pos_spacy':lista_POS, 'token':lista_token})

In [ ]:
df_final = df.merge(df_spacy, on='token', how='left').drop_duplicates().reset_index(drop=True)

In [ ]:
label = {'ADJ':1, 'AUX':2, 'NOUN':3, 'ADP':4, 'DET':5, 'ADV':6, 'VERB':7, 'SCONJ':8, \
         'PROPN':9 , 'NUM':10, 'PUNCT':11, 'CCONJ':12, 'PRON':13, 'SYM':14, 'X':15}

In [ ]:
query = (df_final['pos_spacy'].notnull()) & \
        (df_final['POS_']!='INTJ')

In [ ]:
df_final = df_final[query].reset_index(drop=True)

In [ ]:
df_final['POS_'] = df_final['POS_'].map(label)
df_final['pos_spacy'] = df_final['pos_spacy'].map(label)

In [ ]:
y_true = df_final['POS_']
y_pred = df_final['pos_spacy']


In [ ]:
accuracy_score(y_true, y_pred)

0.8622953184736348

In [ ]:
#accuracy small
#0.8357498376974681

#accuracy large
#0.8622953184736348